In [1]:
from sqlalchemy import create_engine
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv(verbose=True,
            dotenv_path='./.env')
engine = create_engine(f'postgresql://{os.getenv("USERNAME")}:{os.getenv("PASSWORD")}@{os.getenv("HOST")}/{os.getenv("DATABASE")}', 
                       connect_args={'options': '-csearch_path={}'.format('de')})

In [ ]:
%%time
concept = pd.read_sql_table('concept', engine)
condition_occurrence = pd.read_sql_table('condition_occurrence', engine)
# takes about 3 min

# No.2
환자들이 진단 받은 상병 내역 중 첫글자는 (a,b,c,d,e) 문자로 시작하고 중간에
“heart” 단어가 포함된 상병 이름을 찾으려고 합니다.  
condition_occurrence 테이블은
환자가 병원 방문시 진단 받은 질환이 담겨있습니다.  
상병코드는 condition_concept_id이고, concept 테이블의 concept_id와 조인하여 상병 이름을
찾을 수 있습니다. (concept_name 컬럼 사용)
- 문자 검색시 대소문자를 구분하지 않습니다.
- 상병 이름을 중복없이 나열합니다.
  
  
`condition_occurrence`와 `concept`테이블의 concept_id가 (각각 condition_concept_id, concept_id 사용) 같은 것을 기준으로 join한 후 concept_name에서 조건에 맞는 것들을 찾는다.  
pandas의 merge를 활용해서 join 했고 문자열 찾기는 정규식을 활용했다.
  
  ['chronic congestive heart failure']

In [ ]:
condition_occurrence.head()

In [ ]:
concept.head()

In [ ]:
# 필요한 열만 남기기
a = condition_occurrence[['condition_concept_id']]
b = concept[['concept_id', 'concept_name']]

In [ ]:
# 두 테이블 merge
merged_concept = a.merge(b, 'left', left_on='condition_concept_id', right_on='concept_id')

In [ ]:
names = merged_concept['concept_name'].str.lower() # 대소문자 구분하지 않기위해 모두 소문자로

In [ ]:
import re
from collections import defaultdict

answer2 = defaultdict(int)
# a,b,c,d,e로 시작하고 중간에 heart가 들어가는 상병 이름 찾기
comp = re.compile(r'^[abcde].*heart.*')
for n in names:
    if comp.search(n):
        answer2[n] += 1

In [ ]:
answer2.keys() # 정답

In [ ]:
cond = names.str.contains('heart')

In [ ]:
names[cond].value_counts() # heart가 들어가는 상병 이름이 몇 개 없다